# Colab drive mount

In [ ]:
#cd /content/drive/MyDrive/네카라쿠배/최종 프로젝트/code

/content/drive/MyDrive/네카라쿠배/최종 프로젝트/code


In [ ]:
#ls

Collaborative_Filtering_MF_ALS.ipynb  __pycache__/
EDA_sj.ipynb                          set_matplotlib_hangul.py


In [25]:
#!pip install nbformat

# Module import, Pre-Work

In [1]:
from collections import Counter

import numpy as np
import pandas as pd

import scipy.sparse as spr
import pickle

In [2]:
# 결과값 저장하기 위한 write_json 함수 작성
# arena_util.py
# -*- coding: utf-8 -*-

import io
import os
import json
import distutils.dir_util
from collections import Counter

import numpy as np


def write_json(data, fname):
    def _conv(o):
        if isinstance(o, np.int64) or isinstance(o, np.int32):
            return int(o)
        raise TypeError

    parent = os.path.dirname(fname)
    distutils.dir_util.mkpath("../data/" + parent)
    with io.open("../data/" + fname, "w", encoding="utf8") as f:
        json_str = json.dumps(data, ensure_ascii=False, default=_conv)
        f.write(json_str)

'''
def load_json(fname):
    with open(fname, encoding='utf8') as f:
        json_obj = json.load(f)

    return json_obj


def debug_json(r):
    print(json.dumps(r, ensure_ascii=False, indent=4))
'''

# Load Data

In [3]:
ls

Collaborative_Filtering_MF_ALS.ipynb
Collaborative_Filtering_MF_ALS.ipynb.zip


In [4]:
song_meta = pd.read_json("../data/song_meta.json")
train = pd.read_json("../data/train.json")
test = pd.read_json("../data/val.json")

## Data Preprocessing

playlist, song, tag의 id(각각 nid, sid, tid)를 새로 생성하는 이유는, 새로 생성할 id를 matrix의 row, column index로 사용할 것이기 때문입니다.

- plylst_id_nid : playlist id -> nid
- plylst_nid_id : playlist nid -> id
- song_id_sid : song id -> sid
- song_sid_id : song sid -> id
- tag_id_tid : tag id -> tid
- tag_tid_id : tag tid -> id
- song_dict : song id -> count
- tag_dict : tag id -> count

In [5]:
# train + test => new train
plylst = pd.concat([train, test], ignore_index=True)
n_plylst = len(plylst)

# playlist id
plylst["nid"] = range(n_plylst)

# id <-> nid
plylst_id_nid = dict(zip(plylst["id"],plylst["nid"]))
plylst_nid_id = dict(zip(plylst["nid"],plylst["id"]))

In [6]:
plylst_tag = plylst['tags']
tag_counter = Counter([tg for tgs in plylst_tag for tg in tgs])
tag_dict = {x: tag_counter[x] for x in tag_counter}

tag_id_tid = dict()
tag_tid_id = dict()
for i, t in enumerate(tag_dict):
  tag_id_tid[t] = i
  tag_tid_id[i] = t

n_tags = len(tag_dict)

plylst_song = plylst['songs']
song_counter = Counter([sg for sgs in plylst_song for sg in sgs])
song_dict = {x: song_counter[x] for x in song_counter}

song_id_sid = dict()
song_sid_id = dict()
for i, t in enumerate(song_dict):
  song_id_sid[t] = i
  song_sid_id[i] = t

n_songs = len(song_dict)

plylst의 songs와 tags를 새로운 id로 변환하여 DataFrame에 추가합니다

In [7]:
plylst['songs_id'] = plylst['songs'].map(lambda x: [song_id_sid.get(s) for s in x if song_id_sid.get(s) != None])
plylst['tags_id'] = plylst['tags'].map(lambda x: [tag_id_tid.get(t) for t in x if tag_id_tid.get(t) != None])

In [8]:
plylst_use = plylst[['nid','updt_date','songs_id','tags_id']]
plylst_use.loc[:,'num_songs'] = plylst_use['songs_id'].map(len)
plylst_use.loc[:,'num_tags'] = plylst_use['tags_id'].map(len)
plylst_use = plylst_use.set_index('nid')

/Users/jun/miniforge3/envs/ds_study/lib/python3.10/site-packages/pandas/core/indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value


In [9]:
plylst_train = plylst_use.copy()
plylst_train.head()

,updt_date,songs_id,tags_id,num_songs,num_tags
nid,,,,,
0,2013-12-19 18:36:19.000,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",[0],19,1
1,2014-12-02 16:19:42.000,"[19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 3...","[1, 2]",42,2
2,2017-08-28 07:09:34.000,"[61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 7...","[3, 4]",28,2
3,2019-12-05 15:15:18.000,"[89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 1...","[5, 6, 7, 8, 9, 10, 11, 12, 13, 14]",38,10
4,2011-10-25 13:54:56.000,"[127, 128, 129, 130, 131, 132, 133, 134, 135, ...",[15],53,1


## Make Sparse Matrix with scipy.sparse

row가 playlist(nid)이고 column이 item(sid or tid)인 sparse matrix A를 만듭니다.

In [10]:
row = np.repeat(range(n_plylst), plylst_train['num_songs'])
col = [song for songs in plylst_train['songs_id'] for song in songs]
dat = np.repeat(1, plylst_train['num_songs'].sum())
train_songs_A = spr.csr_matrix((dat, (row, col)), shape=(n_plylst, n_songs))

row = np.repeat(range(n_plylst), plylst_train['num_tags'])
col = [tag for tags in plylst_train['tags_id'] for tag in tags]
dat = np.repeat(1, plylst_train['num_tags'].sum())
train_tags_A = spr.csr_matrix((dat, (row, col)), shape=(n_plylst, n_tags))

In [11]:
#전치행렬이 필요없는것으로 판단되어 코드 제거
#train_songs_A_T = train_songs_A.T.tocsr()
#train_tags_A_T = train_tags_A.T.tocsr()

In [12]:
train_songs_A

<138086x638336 sparse matrix of type '<class 'numpy.int64'>'
	with 5707070 stored elements in Compressed Sparse Row format>

In [13]:
1 - (5707070/(138086*638336))

0.9999352538107777

In [15]:
# Sparsity: 얼마나 비어있나?
# Sparsity of plylst x songs matrix
matrix_size = train_songs_A.shape[0]* train_songs_A.shape[1]
num_songs = len(train_songs_A.nonzero()[0])
sparsity = 100 * (1 - (num_songs / matrix_size))
sparsity

99.99352538107777

In [16]:
train_tags_A

<138086x30197 sparse matrix of type '<class 'numpy.int64'>'
	with 503669 stored elements in Compressed Sparse Row format>

In [17]:
1 - (503669/(138086*30197))

0.9998792097797401

In [18]:
# Sparsity: 얼마나 비어있나?
# Sparsity of plylst x tags matrix
matrix_size = train_tags_A.shape[0]* train_tags_A.shape[1]
num_tags = len(train_tags_A.nonzero()[0])
sparsity = 100 * (1 - (num_tags / matrix_size))
sparsity

99.98792097797401

- 첫번째 한계점 발생 : sparsity가 대략 99.5% 이하 수준일 때까지 Collaborative Filtering이 효과가 있는데, 우리의 데이터는 99.9%를 웃돈다.. 

## Make train, test sparse matrix

### 의사 결정의 갈림길
- train set에서 일부를 가려 test set을 생성할 것이다.
- 기존의 val.json의 songs과 tags는 일부가 가려져있다.

- 그렇다면 이 상황에서, 
    - 1. train.json의 일정 비율 만큼 songs들과 tags들만 가릴것이냐? 
    - 2. train+val 의 데이터에서 songs들과 tags들을 일정 비율 함께 가릴것이냐?


- 우선 2번의 방법을 선택 후 진행 => 추후 변경 가능의 여지를 남겨두자

In [19]:
import random
def make_train (matrix, percentage = .2):
    '''
    -----------------------------------------------------
    설명
    유저-아이템 행렬 (matrix)에서 
    1. 0 이상의 값을 가지면 1의 값을 갖도록 binary하게 테스트 데이터를 만들고
    2. 훈련 데이터는 원본 행렬에서 percentage 비율만큼 0으로 바뀜
    
    -----------------------------------------------------
    반환
    training_set: 훈련 데이터에서 percentage 비율만큼 0으로 바뀐 행렬
    test_set:     원본 유저-아이템 행렬의 복사본
    user_inds:    훈련 데이터에서 0으로 바뀐 유저의 index
    '''
    test_set = matrix.copy()
    test_set[test_set !=0] = 1 # binary하게 만들기
    
    training_set = matrix.copy()
    nonzero_inds = training_set.nonzero()
    nonzero_pairs = list(zip(nonzero_inds[0], nonzero_inds[1]))
    
    random.seed(0)
    num_samples = int(np.ceil(percentage * len(nonzero_pairs)))
    samples = random.sample(nonzero_pairs, num_samples)
    
    user_inds = [index[0] for index in samples]
    item_inds = [index[1] for index in samples]
    
    training_set[user_inds, item_inds] = 0
    training_set.eliminate_zeros()
    
    return training_set, test_set, list(set(user_inds))

# 훈련, 테스트 데이터 생성
train_songs_A, test_songs_A, product_users_altered = make_train(train_songs_A, 0.2)
train_tags_A, test_tags_A, product_users_altered = make_train(train_tags_A, 0.2)

# 훈련데이터는 일부분의 1이 0으로 가려지고, 테스트 데이터는 가려지지 않은 데이터
# 훈련데이터로 학습하고 테스트 데이터로 성능을 평가하기 위한 방법

- train_songs_A 와 test_songs_A의 비교

In [20]:
train_songs_A

<138086x638336 sparse matrix of type '<class 'numpy.int64'>'
	with 4565656 stored elements in Compressed Sparse Row format>

In [21]:
test_songs_A

<138086x638336 sparse matrix of type '<class 'numpy.int64'>'
	with 5707070 stored elements in Compressed Sparse Row format>

In [22]:
# train_songs_A 의 Sparsity
1 - (4565656/(138086*638336))

0.9999482030486222

## ALS 구현 (without implicit module)

In [23]:
# from scipy.sparse.linalg import spsolve

In [24]:
'''
def implicit_weighted_ALS(training_set, lambda_val =.1, alpha = 40, n_iter = 10, rank_size = 20, seed = 0):
    '''
#     협업 필터링에 기반한 ALS
#     -----------------------------------------------------
#     input
#     1. training_set : m x n 행렬로, m은 유저 수, n은 아이템 수를 의미. csr 행렬 (희소 행렬) 형태여야 함 
#     2. lambda_val: ALS의 정규화 term. 이 값을 늘리면 bias는 늘지만 분산은 감소. default값은 0.1
#     3. alpha: 신뢰 행렬과 관련한 모수 (C_{ui} = 1 + alpha * r_{ui}). 이를 감소시키면 평점 간의 신뢰도의 다양성이 감소
#     4. n_iter: 반복 횟수
#     5. rank_size: 유저/ 아이템 특성 벡터의 잠재 특성의 개수. 논문에서는 20 ~ 200 사이를 추천하고 있음. 이를 늘리면 과적합 위험성이 있으나 
#     bias가 감소
#     6. seed: 난수 생성에 필요한 seed
#     -----------------------------------------------------
#     반환
#     유저와 아이템에 대한 특성 벡터
    '''
    
    # 1. Confidence matrix
    # C = 1+ alpha * r_{ui}
    conf = (alpha*training_set) # sparse 행렬 형태를 유지하기 위해서 1을 나중에 더함
    
    num_user = conf.shape[0]
    num_item = conf.shape[1]

    # X와 Y 초기화
    rstate = np.random.RandomState(seed)
    X = spr.csr_matrix(rstate.normal(size = (num_user, rank_size)))
    Y = spr.csr_matrix(rstate.normal(size = (num_item, rank_size)))
    X_eye = spr.eye(num_user)
    Y_eye = spr.eye(num_item)
    
    # 정규화 term: 𝝀I
    lambda_eye = lambda_val * spr.eye (rank_size)
    
    # 반복 시작
    for i in range(n_iter):
        yTy = Y.T.dot(Y)
        xTx = X.T.dot(X)
        
        # Y를 고정해놓고 X에 대해 반복
        # Xu = (yTy + yT(Cu-I)Y + 𝝀I)^{-1} yTCuPu
        for u in range(num_user):
            conf_samp = conf[u,:].toarray() # Cu
            pref = conf_samp.copy()
            pref[pref!=0] = 1
            # Cu-I: 위에서 conf에 1을 더하지 않았으니까 I를 빼지 않음 
            CuI = spr.diags(conf_samp, [0])
            # yT(Cu-I)Y
            yTCuIY = Y.T.dot(CuI).dot(Y)
            # yTCuPu
            yTCupu = Y.T.dot(CuI+Y_eye).dot(pref.T)
            
            X[u] = spsolve(yTy + yTCuIY + lambda_eye, yTCupu)
        
        # X를 고정해놓고 Y에 대해 반복
        # Yi = (xTx + xT(Cu-I)X + 𝝀I)^{-1} xTCiPi
        for i in range(num_item):
            conf_samp = conf[:,i].T.toarray()
            pref = conf_samp.copy()
            pref[pref!=0] = 1
            
            #Ci-I
            CiI = spr.diags (conf_samp, [0])
            # xT(Ci-I)X
            xTCiIX = X.T.dot(CiI).dot(X)
            # xTCiPi
            xTCiPi = X.T.dot(CiI+ X_eye).dot(pref.T)
            
            Y[i] = spsolve(xTx + xTCiIX + lambda_eye, xTCiPi)
            
        return X, Y.T
'''

IndentationError: unexpected indent (2464828638.py, line 17)

- 1회 반복 수행해보기
    - lambda = 0.1, alpha = 30, latent dimension = 100

In [25]:
# user_vecs, item_vecs = implicit_weighted_ALS(train_songs_A, lambda_val = 0.1, alpha = 30, n_iter = 1, rank_size = 100)

- 계산량이 상당히 많아서 그런지, 1회 반복임에도 오래걸렸다. (~분)
- 이를 해결하기 위해 implicit module을 이용한다고 한다.
    - 병렬처리가 가능

- 첫번째 플레이리스트의 벡터 확인해보기

In [26]:
# first = user_vecs[0].dot(item_vecs).toarray() # 1x
# first[0,:5]

## ALS구현 (with implicit module)

In [27]:
#pip install --upgrade pip setuptools wheel

In [28]:
#!pip install implicit

In [29]:
import implicit

- alpha 값이 parameter로 지정 할 수 없기 때문에, 따로 지정해줘서 sparse matrix와 곱한 결과 인자로 넣어줘야함 (train_songs_A * alpha)
- sparse matrix의 자료형이 double이어야함 (상관 없는 것 같다)

In [39]:
#Building the model - plylst x song
model = implicit.als.AlternatingLeastSquares(factors=200, regularization=0.1, iterations=1, random_state=13)
alpha_val = 30
data_conf = (train_songs_A * alpha_val) # double이나 int나 거의 비슷한 결과
model.fit(data_conf.T)

  0%|          | 0/1 [00:00<?, ?it/s]

In [40]:
#Building the model - plylst x song
model2 = implicit.als.AlternatingLeastSquares(factors=30, regularization=0.1, iterations=1, random_state=13)
alpha_val = 30
data_conf = (train_tags_A * alpha_val) # double이나 int나 거의 비슷한 결과
model2.fit(data_conf.T)

  0%|          | 0/1 [00:00<?, ?it/s]

### 모델 만드는 함수 build_model, 모델 적합 함수 fit_model 만들기

In [30]:
def build_model(factors=200, regularization=0.1, iteration=100, random_state=13):
    model = implicit.als.AlternatingLeastSquares(factors=factors, regularization=regularization, iterations=iteration, random_state=random_state)
    
    return model

def fit_model(model, ui_sparse_matrix, alpha = 40):
    #global model
    data_conf = (ui_sparse_matrix * alpha)
    model.fit(data_conf.T)

- 하이퍼 파라미터 종류
```
alpha : 신뢰 행렬 만들 때의 파라미터, 플레이리스트에 노래의 존재 여부에 대한 스케일링 term이고 평점간의 신뢰도의 다양성과 정비례
factors : 잠재행렬의 차원의 수 (줄이고 싶은 차원의 수)
regularization : The regularization factor to use
iterations : 반복 수
random_state : 초기 값 설정시 random seed
```

- song_model : playlist와 song에 대한 학습
- tag_model : playlist와 tag에 대한 학습

### 곡 추천 함수 만들기

### def recommend(nid):
- 찾고자 하는 플레이리스트의 nid를 받는다
- 미리 학습한 모델을 기반으로 300개와 30개의 tag를 추천해준다
- 그 중 존재 하지 않는 곡들과 tag들에 한해 추천 목록에 추가해준다 

In [32]:
def recommend(nid, song_model, tag_model):
    rec_sid = song_model.recommend(nid,test_songs_A, N = 300)
    rec_tid = tag_model.recommend(nid,test_tags_A, N = 30)
    
    exist_sid = plylst_train.loc[nid]['songs_id']
    exist_tid = plylst_train.loc[nid]['tags_id']
    
    final_rec_songs = []
    final_rec_tags = []
    
    for sid, score in rec_sid:
        if sid not in exist_sid:
            final_rec_songs.append(song_sid_id[sid])
            if len(final_rec_songs) == 100:
                break
    for tid, score in rec_tid:
        if tid not in exist_tid:
            final_rec_tags.append(tag_tid_id[tid])
            if len(final_rec_tags) == 10:
                break
    
    return plylst_nid_id[nid], final_rec_songs, final_rec_tags
    

In [33]:
def recommend_songs(nid, song_model):
    rec_sid = song_model.recommend(nid,test_songs_A, N = 300)
    
    exist_sid = plylst_train.loc[nid]['songs_id']
    
    final_rec_songs = []
    
    for sid, score in rec_sid:
        if sid not in exist_sid:
            final_rec_songs.append(song_sid_id[sid])
            if len(final_rec_songs) == 100:
                break
    
    return plylst_nid_id[nid], final_rec_songs
    

In [34]:
def recommend_tags(nid, tag_model):
    rec_tid = tag_model.recommend(nid,test_tags_A, N = 30)
    
    exist_tid = plylst_train.loc[nid]['tags_id']
    
    final_rec_tags = []
    
    for tid, score in rec_tid:
        if tid not in exist_tid:
            final_rec_tags.append(tag_tid_id[tid])
            if len(final_rec_tags) == 10:
                break
    return plylst_nid_id[nid], final_rec_tags
    

### recommend_ver2 : tag 예측 성능이 좋지 못하여서 개선한 함수
- 1. 적합이 완료된 song_model을 이용하여 similar한 playlists 30개를 받아온다.
- 2. similar한 순서대로 playlists들의 tag를 가져와서 추가한다
- 3. tag가 10개가 차면 멈춘다.

In [134]:
song_model.similar_users(115071,2)

[(115071, 1.0000001), (72420, 0.94864124)]

In [153]:
#plylst.loc[115071,:]
plylst_train.loc[72420,:]

updt_date                              2015-11-29 12:00:12.000
songs_id     [98384, 130472, 38811, 82767, 73386, 256279, 4...
tags_id                                                   [86]
num_songs                                                   28
num_tags                                                     1
Name: 72420, dtype: object

In [40]:
def recommend_ver2(nid, song_model = model):
    rec_sid = song_model.recommend(nid,test_songs_A, N = 300)
    sim_user_idxs = song_model.similar_users(nid, N = 500)
    
    exist_sid = plylst_train.loc[nid]['songs_id']
    exist_tid = plylst_train.loc[nid]['tags_id']
    
    final_rec_songs = []
    final_rec_tags_tid = []
    
    # 노래 추천
    for sid, score in rec_sid:
        if sid not in exist_sid:
            final_rec_songs.append(song_sid_id[sid])
            if len(final_rec_songs) == 100:
                break
                
    # 태그 추천
    for sim_nid, score in sim_user_idxs[1:]: # tid (중복제외) 10개를 추천 받아보자
        rec_tids = plylst_train.loc[sim_nid,'tags_id']
        for each in rec_tids:
            if (each not in exist_tid): # 기존 플레이리스트에 없다면,
                final_rec_tags_tid.extend(rec_tids) # 추천 플레이리스트(tid로 이루어진)에 넣어라
                final_rec_tags_tid = set(final_rec_tags_tid) # 중복 제거
                final_rec_tags_tid = list(final_rec_tags_tid) # 중복 제거

        if len(final_rec_tags_tid) > 10:
            final_rec_tags_tid = final_rec_tags_tid[:10]
            break
    
    final_rec_tags = [] # tid를 실제 tag이름으로 바꿔주자
    for each in final_rec_tags_tid:
        final_rec_tags.append(tag_tid_id[each])
                        
    return plylst_nid_id[nid], final_rec_songs, final_rec_tags

NameError: name 'model' is not defined

In [36]:
def recommend_tags_ver2(nid, song_model):
    sim_user_idxs = song_model.similar_users(nid, N = 500)
    
    exist_tid = plylst_train.loc[nid]['tags_id']
    
    final_rec_tags_tid = []
                
    for sim_nid, score in sim_user_idxs[1:]: # tid (중복제외) 10개를 추천 받아보자
        rec_tids = plylst_train.loc[sim_nid,'tags_id']
        final_rec_tags_tid.extend(rec_tids)
        final_rec_tags_tid = set(final_rec_tags_tid) # 중복 제거
        final_rec_tags_tid = list(final_rec_tags_tid) # 중복 제거
        if len(final_rec_tags_tid) > 10:
            final_rec_tags_tid = final_rec_tags_tid[:10]
            break
    
    final_rec_tags = [] # tid를 실제 tag이름으로 바꿔주자
    for each in final_rec_tags_tid:
        final_rec_tags.append(tag_tid_id[each])
                
    return plylst_nid_id[nid], final_rec_tags

### model 생성 및 fit

In [96]:
song_model = build_model(factors = 1000, iteration = 30)
tag_model = build_model(factors = 30, iteration = 30)

In [93]:
song_model, tag_model

(<implicit.cpu.als.AlternatingLeastSquares at 0x316ddfac0>,
 <implicit.cpu.als.AlternatingLeastSquares at 0x316ddf280>)

In [97]:
fit_model(song_model, test_songs_A, alpha = 100)
fit_model(tag_model, test_tags_A, alpha = 80)

  0%|          | 0/30 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [107]:
song_model.user_factors.shape, song_model.item_factors.shape

((138086, 1000), (638336, 1000))

In [106]:
np.save('../data/user_factors_1000.npy', song_model.user_factors)
np.save('../data/item_factors_1000.npy', song_model.item_factors)

In [4]:
tmp_user_latent = np.load('../data/user_factors_1000.npy')
tmp_item_latent = np.load('../data/item_factors_1000.npy')

In [21]:
tmp_user_latent[0] * tmp_item_latent[0], 1

(array([ 8.44102644e-04,  6.32819079e-04,  1.24787941e-04, -6.38371101e-04,
        -7.57455360e-04,  1.19899618e-04,  2.92325072e-04, -1.83505553e-03,
        -7.44962716e-04, -9.97394207e-04,  5.91474993e-04,  4.65681340e-04,
         1.32643720e-04, -3.82996484e-04,  4.01481707e-03, -4.37749201e-04,
        -4.89285274e-04, -1.50183114e-05, -1.53524382e-03,  4.31334396e-04,
        -9.32455703e-04,  3.09924100e-04,  1.96873187e-03,  1.55724512e-04,
         1.16741960e-03, -3.92708171e-04, -1.14094280e-03, -4.38030111e-05,
         8.51217424e-04, -3.55470402e-04,  6.00370549e-05,  5.40069828e-04,
        -2.55026360e-04, -3.72729613e-04,  7.33271241e-04, -6.82142505e-04,
         6.51493086e-04, -2.99971198e-05,  1.36006426e-03,  8.51108518e-04,
         1.13197125e-03, -8.79247000e-06,  1.23709231e-03,  6.52832910e-04,
        -2.72557169e-04, -7.85632437e-05,  5.86035568e-03,  9.55441676e-04,
         5.55138744e-04,  2.18311697e-03,  3.79450154e-03, -2.97752937e-04,
        -6.7

In [115]:
sum(song_model.user_factors[0] * song_model.item_factors[67])

0.006928752437286079

In [1]:
tmp_list = []
for i in tqdm(range(5)):
    for j in range(5):
        tmp_list.append(song_model.user_factors[i] * song_model.item_factors[j])
        
tmp_list

NameError: name 'tqdm' is not defined

In [127]:
np.array(tmp_list)

(25, 600)

In [91]:
tmp_list[0].

array([ 5.95174439e-04,  6.42766291e-03,  1.06232159e-03,  1.02544436e-03,
       -9.99996555e-04,  5.61277557e-04, -4.48000181e-04,  9.96891642e-04,
        2.90869200e-03, -1.41763687e-03,  1.30380495e-05, -3.99104203e-04,
        3.84083530e-03, -3.35479825e-04,  9.92288115e-04, -7.96111533e-04,
       -1.12479681e-03, -5.30454214e-04, -1.64472222e-04,  1.62770972e-03,
       -9.03123931e-04, -4.98802256e-05,  5.75515023e-03,  5.88239694e-04,
        2.32572856e-05,  2.62792513e-04,  4.84797201e-04, -2.42970549e-04,
       -2.22542277e-03, -1.24969229e-05,  7.41331186e-03, -1.05955834e-04,
        9.77585209e-04,  1.04878312e-02, -1.25392078e-04,  6.57807977e-04,
        2.50810105e-03, -9.76124313e-04, -4.09618311e-04,  6.71610609e-03,
        7.74393848e-04,  1.31843309e-03,  1.07363936e-04,  1.81941068e-04,
       -2.23457129e-04,  6.10668620e-04,  3.68674565e-03,  4.14789174e-05,
        4.63740085e-04,  1.65977079e-04,  2.94318015e-04,  1.01008883e-03,
        3.63393943e-03,  

### 플레이리스트에 곡과 태그 추천해주기

In [318]:
from tqdm import tqdm
answer = []
for idx in tqdm(range(115071,138086)): # test nid start : 115071
    result = recommend(idx, song_model, tag_model)
    answer.append({
        'id' : result[0],
        'songs' : result[1],
        'tags' : result[2]
    })

 51%|██████████████████▊                  | 11672/23015 [05:48<05:38, 33.51it/s]


KeyboardInterrupt: 

In [316]:
# tag 하이퍼 파라미터만 변경해서 따로 작업했을 때

for idx in tqdm(range(115071,138086)):
    tag_result = recommend_tags_ver2(idx, song_model)
    answer[idx-115071]['tags'] = tag_result[1]

  0%|                                                 | 0/23015 [00:00<?, ?it/s]


IndexError: list index out of range

In [101]:
# simillar user 이용해서 tag 추천하는 recommend_ver2로 전체 결과 추천할 때

from tqdm import tqdm
answer = []
for idx in tqdm(range(115071,138086)): # test nid start : 115071
    result = recommend_ver2(idx, song_model)
    answer.append({
        'id' : result[0],
        'songs' : result[1],
        'tags' : result[2]
    })

100%|█████████████████████████████████████| 23015/23015 [25:21<00:00, 15.12it/s]


In [102]:
# 노래 추천 잘 됐는지 확인
len_wrong_cnt = 0
len_unique_wrong_cnt = 0
wrong_idx = []
for i in range(len(answer)):
    if len(answer[i]['songs']) != 100:
        len_wrong_cnt += 1
        wrong_idx.append({i : len(answer[i]['songs'])})
        
len(answer), len_wrong_cnt, wrong_idx

(23015, 0, [])

In [103]:
# 태그 추천 잘 됐는지 확인
len_wrong_cnt = 0
len_unique_wrong_cnt = 0
wrong_idx = []
for i in range(len(answer)):
    if len(answer[i]['tags']) != 10:
        len_wrong_cnt += 1
        wrong_idx.append({i : len(answer[i]['tags'])})
        
len(answer), len_wrong_cnt, wrong_idx

(23015, 0, [])

In [345]:
plylst.iloc[115071,:]

tags                                                           []
id                                                         118598
plylst_title                                                     
songs           [373313, 151080, 275346, 696876, 165237, 52593...
like_cnt                                                     1675
updt_date                                 2019-05-27 14:14:33.000
nid                                                        115071
songs_id        [82770, 73350, 49850, 283466, 38811, 14654, 83...
tags_id                                                        []
Name: 115071, dtype: object

In [104]:
answer[0]

{'id': 118598,
 'songs': [207912,
  322215,
  623047,
  690767,
  516376,
  472374,
  157900,
  638488,
  78983,
  396028,
  422438,
  421833,
  447762,
  39436,
  382962,
  416417,
  557956,
  703323,
  439161,
  209622,
  217622,
  569715,
  607459,
  182907,
  411438,
  606063,
  219415,
  328908,
  307938,
  259770,
  187531,
  66475,
  212592,
  249378,
  367826,
  140837,
  448547,
  226991,
  412246,
  693988,
  545816,
  413459,
  169426,
  545553,
  485776,
  262430,
  551157,
  297376,
  616144,
  467269,
  620311,
  438857,
  554509,
  394031,
  646988,
  10690,
  585728,
  380092,
  292859,
  654428,
  680970,
  455806,
  584108,
  209684,
  530762,
  590972,
  220100,
  343420,
  626157,
  251035,
  242188,
  546070,
  650957,
  21512,
  385871,
  193674,
  546713,
  444103,
  267837,
  404536,
  667791,
  407255,
  648022,
  498401,
  419274,
  375776,
  283939,
  494552,
  320642,
  271335,
  352807,
  225652,
  462968,
  151704,
  26908,
  70465,
  574308,
  693580,
  1

### 결과물 생성

- write_json : json파일 저장 함수

In [45]:
import io
import os
import json
import distutils.dir_util
from collections import Counter

import numpy as np


def write_json(data, fname):
    def _conv(o):
        if isinstance(o, np.int64) or isinstance(o, np.int32):
            return int(o)
        raise TypeError

    parent = os.path.dirname(fname)
    distutils.dir_util.mkpath("../data/" + parent)
    with io.open("../data/" + fname, "w", encoding="utf8") as f:
        json_str = json.dumps(data, ensure_ascii=False, default=_conv)
        f.write(json_str)

In [105]:
write_json(answer, "./similar_user/f_1000/results.json")

---

---

##### 함수 만드는 과정에서 썼던 테스트 코드들

- 2번 nid를 가진 playlist의 맞춤 태그를 추천해보자

In [129]:
rec_tid = model2.recommend(2,train_tags_A, N = 30)
rec_tid[:10]

[(112, 0.22263646),
 (72, 0.108597435),
 (65, 0.083253555),
 (103, 0.07775032),
 (417, 0.07139853),
 (184, 0.07092448),
 (71, 0.063434176),
 (111, 0.062599815),
 (4509, 0.054990232),
 (437, 0.05413252)]

In [130]:
for tid,_ in rec_tid:
    print(tag_tid_id[tid])

매장음악
비오는날
가을
겨울
커피
매장
설렘
여름
까페에서듣기좋은노래
따뜻한
봄
차분한
공감
음색
우울할때
산책
상시업뎃
EDM
새벽감성
감성곡
출근길
새해
공부
별
브런치
홈카페
한강
리얼북
어쿠스틱
sweetmind


- 2번(nid)플레이리스트에 추천해줄만한 곡을 상위 10개만 찾아보자

In [131]:
rec_sid = model.recommend(2,train_songs_A, N = 300)
rec_sid[:10]

[(7178, 0.4791444),
 (42964, 0.46107218),
 (12776, 0.45183596),
 (17659, 0.43752182),
 (5317, 0.43498644),
 (21914, 0.43245193),
 (45992, 0.4268237),
 (16933, 0.4262839),
 (2280, 0.42286178),
 (24641, 0.41935915)]

In [80]:
for sid, score in rec_sid[:10]:
    print(song_sid_id[sid])
    print(song_meta.loc[song_sid_id[sid]]['song_name'])
    print(song_meta.loc[song_sid_id[sid]]['artist_name_basket'])
    print(song_meta.loc[song_sid_id[each]]['song_gn_gnr_basket'])
    print('------------------------')

88403
Psychic (사이킥)
['리차드파커스']
['GN0500', 'GN0600']
------------------------
102004
Perfume
['Nu.D']
['GN0500', 'GN0600']
------------------------
698965
별 침대 옥상 (With Boni)
['윌콕스']
['GN0500', 'GN0600']
------------------------
691849
Dress Code (Feat. G2)
['윌콕스']
['GN0500', 'GN0600']
------------------------
616212
너를 그리다
['디어 (d.ear)']
['GN0500', 'GN0600']
------------------------
564963
망고
['CHEEZE (치즈)']
['GN0500', 'GN0600']
------------------------
454075
Angel 2 Me (Duet. Jeff Bernat)
['맥케이 (McKay)']
['GN0500', 'GN0600']
------------------------
56335
닮아 (With 슬로 (Sllo), 제이형 Of 타코앤제이형)
['정바스']
['GN0500', 'GN0600']
------------------------
322158
내게 Him을 줘
['Henmi']
['GN0500', 'GN0600']
------------------------
670510
Wanna Love You
['폴킴']
['GN0500', 'GN0600']
------------------------


- 그렇다면 현재 들어있는 2번(nid) 플레이리스트의 노래들을 찾아보자

In [55]:
plylst_nid_id[2] #원래 plylst id = 76951

76951

In [73]:
exist_sid = plylst_train.loc[2]['songs_id'] #담겨있는 songs_sid
len(exist_sid), exist_sid[:5]

(28, [61, 62, 63, 64, 65])

In [79]:
for each in exist_sid:
    print(song_sid_id[each])
    print(song_meta.loc[song_sid_id[each]]['song_name'])
    print(song_meta.loc[song_sid_id[each]]['artist_name_basket'])
    print(song_meta.loc[song_sid_id[each]]['song_gn_gnr_basket'])
    print('------------------------')

83116
도시의 밤
['소울라이츠']
['GN0400']
------------------------
276692
I`m Alright
['소울라이츠']
['GN0400']
------------------------
166267
너를 좋아하니까
['뷰티핸섬']
['GN0500', 'GN0600']
------------------------
186301
247 (Feat. AMJ)
['소울사이어티']
['GN0400']
------------------------
354465
그랬으면 (With 언피)
['카멜']
['GN0100']
------------------------
256598
싶어 (Feat. Reddy)
['오디오라인 (Audioline)']
['GN0400']
------------------------
233195
비숲
['이루펀트', '김박첼라']
['GN0300']
------------------------
666852
오늘 더 좋아
['슈가볼']
['GN0500', 'GN0600']
------------------------
686560
잠든 널 두고 나오며
['슈가볼']
['GN0500', 'GN0600']
------------------------
556426
비행기모드 (Feat. Koonta Of Rude Paper, 오성)
['Common Tale']
['GN0300']
------------------------
142974
Your Song (With 이진아&정승환&권진아)
['샘김 (Sam Kim)']
['GN0800']
------------------------
331878
같았으면해
['와인루프 (Wine Loop)']
['GN0500', 'GN0400']
------------------------
195141
5:30 pm
['김새한길']
['GN0400']
------------------------
32017
사랑인가요 (With 슬로)
['타코앤제이형']
['GN0500', 'GN0300']
---

In [64]:
song_meta.loc[song_sid_id[sid]]

song_gn_dtl_gnr_basket    [GN0401, GN0403]
issue_date                        20170327
album_name                  Wanna Love You
album_id                          10049202
artist_id_basket                  [752425]
song_name                   Wanna Love You
song_gn_gnr_basket                [GN0400]
artist_name_basket                    [폴킴]
id                                  670510
Name: 670510, dtype: object

In [47]:
# 115071 부터 test set에 대한 idx
plylst_nid_id[115071]

118598

---

In [375]:
song_model.similar_users(115071,300)[-5:]

[(87989, 0.33962598),
 (3632, 0.33799744),
 (131579, 0.337948),
 (86842, 0.3364419),
 (121949, 0.33557162)]

In [153]:
#plylst.loc[115071,:]
plylst_train.loc[72420,:]

updt_date                              2015-11-29 12:00:12.000
songs_id     [98384, 130472, 38811, 82767, 73386, 256279, 4...
tags_id                                                   [86]
num_songs                                                   28
num_tags                                                     1
Name: 72420, dtype: object